In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.font_manager import FontProperties
from math import log
import operator
import pickle

## 生成数据集

In [2]:
def createDataSet():
    dataSet = [[0, 0, 0, 0, "no"],
               [0, 0, 0, 1, "no"],
               [0, 1, 0, 1, "yes"],
               [0, 1, 1, 0, "yes"],
               [0, 0, 0, 0, "no"],
               [1, 0, 0, 0, "no"],
               [1, 0, 0, 1, "no"],
               [1, 1, 1, 1, "yes"],
               [1, 0, 1, 2, "yes"],
               [1, 0, 1, 2, "yes"],
               [2, 0, 1, 2, "yes"],
               [2, 0, 1, 1, "yes"],
               [2, 1, 0, 1, "yes"],
               [2, 1, 0, 2, "yes"],
               [2, 0, 0, 0, "no"]]
    labels = ["F1", "F2", "F3", "F4"]
    return dataSet, labels

## 得到该classList最多的类型yes/no

In [3]:
def majorityCnt(classList):
    '''
    return: 最多的类型（yes/no）
    '''
    classCount = {}
    for vote in classCount:
        if vote not in classCount.Keys():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    #print(sortedClassCount)
    return sortedClassCount[0][0]

## 得到当前数据中最好的特征索引

In [4]:
def chooseBestFeatureToSplit(dataset):
    '''
    return: 当前数据中最好的特征索引
    dataset: 一般为被切过的数据
    '''
    #一般为裁剪后的列数总和
    numFeatures = len(dataset[0]) - 1
    
    #基础熵值
    baseEntropy = calcShannonEnt(dataset)
    #最好的信息增益
    bestInfoGain = 0
    #最好的特征
    bestFeature = -1
    for i in range(numFeatures):
        #得到当前列特征值
        featList = [example[i] for example in dataset]
        uniqueVals = set(featList)
        #分别计算以该列特征分类后的熵值
        newEntropy = 0
        for val in uniqueVals:
            #分别计算以该列特征分类后的每组结点的熵值
            subDataSet = splitDataSet(dataset, i, val)
            #计算加权熵值
            prob = len(subDataSet) / float(len(dataset))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i
    
    return bestFeature

## 计算香农熵

In [5]:
def calcShannonEnt(dataset):
    '''
    func: 计算香农熵
    dataset: 一般为处理后的部分数据
    '''
    numSamples = len(dataset)
    labelCounts = {}
    for feaVec in dataset:
        currentlabel = feaVec[-1]
        if currentlabel not in labelCounts.keys():
            labelCounts[currentlabel] = 0
        labelCounts[currentlabel] += 1
    shannonEnt = 0
    for key in labelCounts:
        #prop key类别的概率值
        prob = float(labelCounts[key]) / numSamples
        shannonEnt -= prob * log(prob, 2)
    return shannonEnt

## 切分数据集

In [6]:
def splitDataSet(dataset, axis, val):
    '''
    func: 按该特征(axis)分类后该结点(val)的数据切分数据集，
    并去掉当前特征列，返回dataset长宽都缩减后的结果
    axis: 当前特征索引
    val: 当前特征的当前结点所代表的值，如weather特征中的“cloud”
    '''
    retDataSet = []
    for featVec in dataset:
        if featVec[axis] == val:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

## 主要函数---生成决策树

In [7]:
def createTree(dataset, labels, featlabels):
    classList = [example[-1] for example in dataset]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataset[0]) == 1:
        return majorityCnt(classList)
    #得到当前最好特征的索引
    bestFeat = chooseBestFeatureToSplit(dataset)
    bestFeatLabel = labels[bestFeat]
    #按顺序拿到label
    featLabels.append(bestFeatLabel)
    
    myTree = {bestFeatLabel: {}}
    del labels[bestFeat]
    
    #拿到当前特征有几种值，即能分为几个分支
    featValue = [example[bestFeat]for example in dataset]
    uniqueVals = set(featValue)
    for value in uniqueVals:
        #当前label已经被切割
        sublabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataset, bestFeat, value),
                                                 sublabels, featLabels)
    return myTree

## 可视化

## 测试代码

In [8]:
dataset, labels = createDataSet()
featLabels = []
myTree = createTree(dataset, labels, featLabels)